In [4]:
from termcolor import colored
from tqdm import tqdm
import pandas as pd

data = pd.read_csv('./data/evaluation_data.csv')

questions = data["question"].to_list()

In [5]:
questions

['What is the deadline for transfer students to apply for the Fall Semester?',
 'Who is authorized to request verification of identity at SUNY Brockport?',
 'What does a probationary appointment at SUNY Brockport entail?',
 'What characteristics do you seek in a Peer Mentor?',
 'What are some effective strategies for preparing for an interview at SUNY Brockport?',
 'What are the eligibility requirements for the SUNY award nomination at Brockport?',
 'What kind of clinical experience will I get during the Athletic Training program?',
 "Are there any situations where SUNY Brockport's policy on reimbursement for moving expenses does not apply?",
 'What is the schedule for proposal evaluation by the IRB at SUNY Brockport?',
 'How can I gather more information on the sociology program and tour the SUNY Brockport campus?',
 'Can all students participate in the international field experience?',
 'What degrees are offered in Physical Education Teacher Education at SUNY Brockport?',
 "I'm inter

In [ ]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # prevent tensorflow logs

# Set path to parent directory so we can import from other folders.
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from fine_tuning.finetune_class import FineTunedModel
from scratch_model.inference import ScratchModel
from text_search.rag import RAG
from termcolor import colored
from tqdm import tqdm
import pandas as pd

# ----- Loading in models/data -----

data = pd.read_csv('./data/evaluation_data.csv')
questions = data["question"].to_list()[:10]


# ---------------------------------------------------------------------
# RAG
print("Beginning RAG evaluation...")
model = RAG(
    main_categorization_model_dir="../text_search/models/main_category",
    subcategorization_model_dir="../text_search/models/subcategory_models/",
    embeddings_file="../text_search/data/embeddings.pickle"
)

answers = []
for question in tqdm(questions):
    answer = model(
        question=question,
        config={
            "search_type": "typesense",
            "use_classifier": True,
            "n_results": 5,
            "model_kwargs": {"temperature": 0.0},
        }
    )
    answers.append(answer)

df = pd.DataFrame({"question": questions, "answer": answers})
df.to_csv("data/rag_evaluation_answers.csv", index=False)

del model


# ---------------------------------------------------------------------
# Finetuned 
print("Beginning FineTuned evaluation...")
model = FineTunedModel(repo_id="msaad02/BrockportGPT-7b")

answers = []
for question in tqdm(questions):
    answers.append(model(question, temperature=0))

del model

df = pd.DataFrame({"question": questions, "answer": answers})
df.to_csv("data/finetuned_evaluation_answers.csv", index=False)


# ---------------------------------------------------------------------
# Scratch
print("Beginning Scratch evaluation...")

# Breaking things up into chunks to prevent memory errors
n = 2
questions_split = [questions[i * n:(i + 1) * n] for i in range((len(questions) + n - 1) // n )]

answers_total = []
with tqdm(total=len(questions)) as pbar:
    for id, questions_chunk in enumerate(questions_split):
        model = ScratchModel(model_dir="../scratch_model/models/transformer_v7/")

        answers_chunk = []
        for question in questions_chunk:
            answers_chunk.append(model(question))
            answers_total.append(model(question))
            pbar.update()

        df = pd.DataFrame({"question": questions_chunk, "answer": answers_chunk})
        df.to_csv(f"data/scratch_parts/scratch_evaluation_answers_{id}.csv", index=False)

        del model

if len(answers_total) != len(questions):
    raise ValueError("Number of answers does not match number of questions")
else:
    df = pd.DataFrame({"question": questions, "answer": answers_total})
    df.to_csv("data/scratch_evaluation_answers.csv", index=False)

In [ ]:
# RAG
# model = RAG()

In [7]:
n=5
[questions[i * n:(i + 1) * n] for i in range((len(questions) + n - 1) // n )]  

[['What is the deadline for transfer students to apply for the Fall Semester?',
  'Who is authorized to request verification of identity at SUNY Brockport?',
  'What does a probationary appointment at SUNY Brockport entail?',
  'What characteristics do you seek in a Peer Mentor?',
  'What are some effective strategies for preparing for an interview at SUNY Brockport?'],
 ['What are the eligibility requirements for the SUNY award nomination at Brockport?',
  'What kind of clinical experience will I get during the Athletic Training program?',
  "Are there any situations where SUNY Brockport's policy on reimbursement for moving expenses does not apply?",
  'What is the schedule for proposal evaluation by the IRB at SUNY Brockport?',
  'How can I gather more information on the sociology program and tour the SUNY Brockport campus?'],
 ['Can all students participate in the international field experience?',
  'What degrees are offered in Physical Education Teacher Education at SUNY Brockport?